In [4]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
import random

In [6]:
def Day1(data=get_input(1, 2021)):
  def p1():
    pass
  def p2():
    pass

  return p1(),p2(),data

Day1()

(None,
 None,
 ["Please don't repeatedly request this endpoint before it unlocks! The calendar countdown is synchronized with the server time; the link will be enabled on the calendar the instant this puzzle becomes available."])

In [ ]:
#aoc_utils.submit_answer(1, 2021, 1, _[0])
#aoc_utils.submit_answer(1, 2021, 2, _[1])